# 🎮 Corona Control Ultimate — Cloud GPU Desktop

Dieses Notebook richtet einen **Remote Desktop mit NVIDIA T4 GPU** ein.
Dein Laptop-GPU wird **0% belastet** — alles rendert auf Googles Hardware!

## Anleitung:
1. ⬆️ Oben: `Runtime` → `Change runtime type` → **T4 GPU** auswählen
2. ▶️ Jede Zelle der Reihe nach ausführen (Play-Button links)
3. 🔗 Am Ende bekommst du einen Link zum Remote Desktop
4. 🎮 Im Remote Desktop: Chrome öffnet automatisch dein Spiel!

In [ ]:
#@title 📋 Schritt 1: GPU prüfen
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)
if 'T4' in result.stdout or 'V100' in result.stdout or 'A100' in result.stdout:
    print('✅ GPU erkannt! Weiter mit Schritt 2.')
else:
    print('❌ Keine GPU! Gehe zu Runtime → Change runtime type → T4 GPU')

In [ ]:
#@title 🖥️ Schritt 2: Desktop-Umgebung installieren (dauert ~2 Min)
%%bash
apt-get update -qq > /dev/null 2>&1
apt-get install -y -qq xfce4 xfce4-goodies xvfb x11vnc novnc websockify > /dev/null 2>&1
echo '✅ Desktop-Umgebung installiert!'

In [ ]:
#@title 🌐 Schritt 3: Google Chrome installieren
%%bash
wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
apt-get install -y -qq ./google-chrome-stable_current_amd64.deb > /dev/null 2>&1
rm google-chrome-stable_current_amd64.deb
echo '✅ Google Chrome installiert!'

In [ ]:
#@title 🔧 Schritt 4: GPU-Treiber für WebGL konfigurieren
%%bash
# Vulkan support für WebGL
apt-get install -y -qq mesa-vulkan-drivers libvulkan1 vulkan-tools > /dev/null 2>&1

# EGL/GLX support
apt-get install -y -qq libegl1-mesa libgl1-mesa-glx libgl1-mesa-dri > /dev/null 2>&1

echo '✅ GPU-Treiber für WebGL konfiguriert!'

In [ ]:
#@title 🚀 Schritt 5: Remote Desktop starten + Spiel öffnen
import subprocess
import time
import os
from google.colab.output import eval_js

# Virtuelle Anzeige starten
display = subprocess.Popen([
    'Xvfb', ':99', '-screen', '0', '1920x1080x24', '-ac'
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
os.environ['DISPLAY'] = ':99'
time.sleep(2)

# XFCE Desktop starten
desktop = subprocess.Popen([
    'startxfce4'
], env={**os.environ}, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(3)

# VNC Server starten
vnc = subprocess.Popen([
    'x11vnc', '-display', ':99', '-forever', '-nopw', '-shared', '-rfbport', '5900'
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(2)

# noVNC WebSocket Bridge starten
novnc = subprocess.Popen([
    'websockify', '--web', '/usr/share/novnc/', '6080', 'localhost:5900'
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(2)

# Chrome mit WebGL + GPU öffnen
chrome = subprocess.Popen([
    'google-chrome',
    '--no-sandbox',
    '--disable-dev-shm-usage',
    '--use-angle=vulkan',
    '--enable-features=Vulkan',
    '--enable-gpu-rasterization',
    '--enable-webgl',
    '--ignore-gpu-blocklist',
    '--start-maximized',
    'https://wrzzzrzr-coronatraeneu.hf.space/'
], env={**os.environ}, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

print('✅ Remote Desktop läuft!')
print('✅ Chrome öffnet automatisch Corona Control Ultimate!')
print()
print('='*60)
print('🔗 DEIN REMOTE DESKTOP LINK:')
print('='*60)

# noVNC URL über Colab-Proxy erstellen
try:
    from google.colab.output import eval_js
    url = eval_js("google.colab.kernel.proxyPort(6080, {'cache': false})")
    print(f'\n👉 {url}/vnc.html?autoconnect=true&resize=remote')
except:
    print('\n👉 Klicke oben auf das Colab-Symbol → Port 6080')
    print('   Oder öffne: http://localhost:6080/vnc.html?autoconnect=true')

print()
print('🎮 Im Remote Desktop siehst du Chrome mit deinem Spiel!')
print('🖱️ Klicke auf EINSATZ BEGINNEN um loszulegen!')
print('💻 Deine lokale GPU: 0% Last — alles auf Googles T4!')

In [ ]:
#@title 🔍 GPU-Last prüfen (optional)
!nvidia-smi

In [ ]:
#@title ⏰ Session am Leben halten (letzte Zelle, laufen lassen!)
import time
print('Session aktiv halten... (bis zu 12 Stunden)')
print('Schließe diese Zelle NICHT!')
while True:
    time.sleep(300)  # Alle 5 Min ein Heartbeat
    print('.', end='', flush=True)